In [16]:
import panel as pn
import pandas as pd
import hvplot.pandas
import holoviews as hv

In [17]:
pn.extension()

In [18]:
df = pd.read_csv('jobs_cleaned_1.csv')

In [19]:
del(df['Unnamed: 0'])

In [20]:
df.columns

Index(['Job', 'Salary_Begin', 'Salary_End', 'Hourly', 'Monthly', 'Yearly',
       'Company', 'Rating', 'Posted_Date', 'Posted_Month', 'Type', 'Benefits',
       'Qualifications', 'Job Link', 'Job_simp', 'Job_seniority',
       'Location_type', 'num_Qualifications', 'num_Benefits', 'Salary_Average',
       'num_Type', 'Date_Posted_Complete'],
      dtype='object')

In [21]:
Type = pd.read_csv('type.csv')

In [22]:
del(Type['Unnamed: 0'])
Type = Type.rename(columns={'0': 'Type of Job'})

In [34]:
qualifications = pd.read_csv('qualifications.csv')

In [35]:
del(qualifications['Unnamed: 0'])
qualifications = qualifications.rename(columns={'0': 'Qualifications'})

In [36]:
benefits = pd.read_csv('benefits.csv')

In [37]:
del(benefits['Unnamed: 0'])
benefits = benefits.rename(columns={'0': 'Benefits'})

In [23]:
column_selector = pn.widgets.Select(value='Rating', options=['Rating', 'Salary_Average', 'Salary_Begin', 'Salary_End'], align = ('start', 'end'))

In [24]:
def description(column):
    summary_stats = pd.DataFrame(df[column].describe().apply(lambda x: round(x, 2))).T
    return summary_stats

In [90]:
def create_hist(column):
    histogram = df.hvplot.hist(y=column, width=450, xlabel='', color='#47EF26')
    
    # Hide the Bokeh logo
    hist_plot = hv.render(histogram, backend='bokeh')
    hist_plot.output_backend = 'webgl'  # Set the output backend
    hist_plot.toolbar.logo = None  # Hide the Bokeh logo
    
    # Show the plot
    return hist_plot

In [92]:
def create_box(column):
    box = df[column].hvplot.box(width=450, ylabel='', tools=['hover'], color='#47EF26')
    
    # Hide the Bokeh logo
    box_plot = hv.render(box, backend='bokeh')
    box_plot.output_backend = 'webgl'  # Set the output backend
    box_plot.toolbar.logo = None  # Hide the Bokeh logo
    
    # Show the plot
    return box_plot

In [27]:
date = {
    'Date': pd.date_range(start='2023-09-22', periods=15, freq='D'),
    'Count': df['Date_Posted_Complete'].value_counts().sort_index().values
}

date_pd = pd.DataFrame(date)

In [28]:
def create_lineplot():
    line = date_pd.hvplot(x='Date', y='Count', width=900)
    
    # Hide the Bokeh logo
    line_plot = hv.render(line, backend='bokeh')
    line_plot.output_backend = 'webgl'  # Set the output backend
    line_plot.toolbar.logo = None  # Hide the Bokeh logo
    
    # Show the plot
    return line_plot

In [95]:
column_selector_bar = pn.widgets.Select(options=['Job_simp', 'Company', 'Type of Job', 'Qualifications', 'Benefits'], value='Type of Job', align = ('start', 'end'), width=110, margin=(12,0,0,0))

range_slider_bar = pn.widgets.IntRangeSlider(name='Index Range', start=0, end=9, value=(0,5), margin=(12,0,0,10))

def set_range(event):
    if column_selector_bar.value=='Type of Job':
        range_slider_bar.end=len(Type[column_selector_bar.value].value_counts())
    elif column_selector_bar.value=='Qualifications':
        range_slider_bar.end=len(qualifications[column_selector_bar.value].value_counts())
    elif column_selector_bar.value=='Benefits':
        range_slider_bar.end=len(benefits[column_selector_bar.value].value_counts())
    else:
        range_slider_bar.end=len(df[column_selector_bar.value].value_counts())

column_selector_bar.param.watch(set_range, 'value')

def create_bar(column, ranges):
    if column=='Type of Job':
        data=Type[column].value_counts()[ranges[0]:ranges[1]]
    elif column=='Qualifications':
        data=qualifications[column].value_counts()[ranges[0]:ranges[1]]
    elif column=='Benefits':
        data=benefits[column].value_counts()[ranges[0]:ranges[1]]
    else:
        data=df[column].value_counts()[ranges[0]:ranges[1]]
    
    bar = data.reset_index().hvplot.barh(x=column, width=450, xlabel='', ylabel='', color='#EF26A0')
    
    # Hide the Bokeh logo
    bar_plot = hv.render(bar, backend='bokeh')
    bar_plot.output_backend = 'webgl'  # Set the output backend
    bar_plot.toolbar.logo = None  # Hide the Bokeh logo
    
    # Show the plot
    return bar_plot

In [70]:
search_box = pn.widgets.TextInput(value='python')

def database(search_term):
    # Convert the search term to lowercase for case-insensitive search
    search_term = search_term.lower()
    
    # Use apply to check if the search term is in 'Qualifications' or 'Benefits' for each row
    mask = df.apply(lambda row: search_term in str(row['Qualifications']).lower() or search_term in str(row['Benefits']).lower(), axis=1)
    
    # Filter the DataFrame based on the condition
    filtered_df = df[mask]
    
    # Select specific columns
    selected_columns = ['Job', 'Salary_Begin', 'Salary_End', 'Salary_Average', 'Company', 'Rating', 'Posted_Date', 'Posted_Month', 'Type', 'Benefits', 'Qualifications']
    
    return filtered_df[selected_columns]

In [96]:
pn.Column(
    pn.Row(column_selector, pn.pane.DataFrame(pn.bind(description,column_selector), width=568), pn.Spacer(width=50), search_box),
    pn.Row(pn.panel(pn.bind(create_hist, column_selector)), pn.panel(pn.bind(create_box, column_selector)), pn.Spacer(width=50), pn.pane.DataFrame(pn.bind(database, search_box), width=450, height=330, margin=(-40,0,0,0))),
    pn.Row(pn.Spacer(width=950), column_selector_bar, range_slider_bar),
    pn.Row(pn.panel(create_lineplot), pn.Spacer(width=50), pn.panel(pn.bind(create_bar, column_selector_bar, range_slider_bar)))
).servable()

Column
    [0] Row
        [0] Select(align=('start', 'end'), options=['Rating', 'Salary_Average...], value='Rating')
        [1] DataFrame(DataFrame, width=568)
        [2] Spacer(width=50)
        [3] TextInput(value='java', value_input='java')
    [1] Row
        [0] ParamFunction(function, _pane=Bokeh, defer_load=False)
        [1] ParamFunction(function, _pane=Bokeh, defer_load=False)
        [2] Spacer(width=50)
        [3] DataFrame(DataFrame, height=330, margin=(-40, 0, 0, 0), sizing_mode='fixed', width=450)
    [2] Row
        [0] Spacer(width=950)
        [1] Select(align=('start', 'end'), margin=(12, 0, 0, 0), options=['Job_simp', 'Company', ...], value='Type of Job', width=110)
        [2] IntRangeSlider(end=9, margin=(12, 0, 0, 10), name='Index Range', value=(0, 5), value_end=5)
    [3] Row
        [0] Column
            [0] Column()
            [1] Row
                [0] Bokeh(figure, name='interactive43872')
        [1] Spacer(width=50)
        [2] ParamFunction(function, _pane=Bokeh, defer_load=False)